In [16]:
%load_ext autoreload
%autoreload 2
from shot.models import Period, Year, Month, Week, Day, Session, Shot, SonyData, SessionLabel
from video.models import VideoSource
from django.contrib.auth.models import User
from profiles.models import UserProfile
from shot.import_sony import csv2shotsvideos
#from passlib.hash import pbkdf2_sha256
from shot.tasks import sony_csv_to_db
import datetime as dt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
user = User(username='lionardo')
user.save()

<QuerySet [<User: admin>, <User: ales>, <User: lionardo>]>

In [13]:
year = Year.objects.all()[0]
year.delete()

(6453,
 {'shot.Day': 8,
  'shot.Month': 2,
  'shot.Period': 19,
  'shot.Session': 8,
  'shot.Session_labels': 0,
  'shot.Shot': 3190,
  'shot.SonyData': 3190,
  'shot.Year': 1,
  'video.VideoSource': 35})

In [7]:
user = User.objects.get(username='lionardo')
user, user.pk, Year.objects.count(), Month.objects.count(), Week.objects.count(), Day.objects.count(), Session.objects.count(), Shot.objects.count(), SonyData.objects.count(), VideoSource.objects.count()

(<User: lionardo>, 3, 3, 4, 8, 18, 18, 8772, 8772, 45)

In [8]:
shot = user.shots.all()[0]
shot.data.swing_type

'FH'

In [2]:
#pks = sony_csv_to_db('/home/alessandro/20170131214425.csv',3)

In [12]:
shots_videos = csv2shotsvideos('/home/alessandro/Videos/SonySensor/20161228194109.csv')
len(shots_videos)

3225

In [3]:
day = Day.objects.all()[1]
day.videosource_set.count()

7

In [13]:
len(shots_videos[shots_videos['video_filename'].notnull()]), len(shots_videos)

(35, 3225)

In [5]:
day = Day.objects.filter(user=user)[3]
session = day.session_set.all()[0]

In [4]:
label = SessionLabel()
label.name = 'Singles'
label.user = user
label.save()

In [12]:
session.labels.add(label)

In [14]:
label.session_set.add(session)

In [16]:
Session.objects.filter(user=user)

<QuerySet [<Session: 2016/11/22/18>, <Session: 2016/11/23/17>, <Session: 2016/11/28/18>, <Session: 2016/12/1/19>, <Session: 2016/12/2/17>, <Session: 2016/12/5/20>, <Session: 2016/12/7/18>, <Session: 2016/12/28/17>]>

In [102]:
from api.serializers import DaySerializer, LabelSerializer, SessionSerializer

data = {"user":2,"name":"Dingles","slug":"asd"}
ll = LabelSerializer(lab,data=data)
ll.is_valid()

True

In [104]:
ll.validated_data

OrderedDict([('user', <User: ales>), ('name', 'Dingles'), ('slug', 'asd')])

In [93]:
instance.pk

6

In [105]:
lab = SessionLabel.objects.all()[3]
lab.pk, lab.user, lab.name, lab.slug

(4, <User: ales>, 'Dingles', 'dingles')

In [135]:
ss = SessionSerializer(session)
data = ss.data
data

ReturnDict([('pk', 34),
            ('user', 2),
            ('timestamp', '2016-12-01T19:44:37Z'),
            ('shot_count', 570),
            ('video_count', 4),
            ('labels',
             [OrderedDict([('pk', 1),
                           ('user', 2),
                           ('name', 'Singles'),
                           ('slug', 'singles')])])])

In [136]:
data['labels'].append(LabelSerializer(label).data)

In [7]:
shot = day.shots.all()[100]


In [3]:
#days = Day.objects.filter()[0:3]

In [31]:
for day in days:
    print(day, day.pk)

2016/11/22 26
2016/11/23 28
2016/11/28 30


In [95]:
the_days = Day.objects.filter(user=user,pk__in=[26,28,30])
from django.db.models import Q
#QS = []
QS = Q(user__username='admin')
QS.add(Q(day__in=the_days), Q.AND)
#QS.add(Q(user=user), Q.AND)
#que.append(Q(user=user))
QS

SyntaxError: keyword can't be an expression (<ipython-input-95-ac90c04c9c74>, line 5)

In [93]:
Shot.objects.filter(QS).count()

0

In [45]:
from api.objects import SonyFilter
input_dict = {
"username":"ales",
"sensor":"SO",
"filters":{
	"periods":{
		"name":"day",
		"pks":[30,33,35]
		},
	"date_range":{
		"start":"yyyy-mm-dd",
		"end":"yyyy-mm-dd"
		},
	"swing_speed":{
		"min":0,
		"max":1000
		},
	"ball_speed":{
		"min":0,
		"max":1000
		},
	"ball_spin":{
		"min":-10,
		"max":10
		},
	"swing_type":[]
	}
}

In [51]:
obj = SonyFilter(input_dict)

In [53]:
obj.filter

{'filters': {'ball_speed': {'max': 1000, 'min': 0},
  'ball_spin': {'max': 10, 'min': -10},
  'periods': {'name': 'day', 'pks': [30, 33, 35]},
  'swing_speed': {'max': 1000, 'min': 0}},
 'sensor': 'SO',
 'username': 'ales'}

In [110]:
end = dt.datetime.strptime('2016-12-1'+'T23:59:59','%Y-%m-%dT%H:%M:%S')

end


datetime.datetime(2016, 12, 1, 23, 59, 59)

In [5]:
from video.video_routines import process_video_source

In [6]:
user = User.objects.get(username='ales')
video_source = VideoSource.objects.get(pk=166)
user, video_source


(<User: ales>, <VideoSource: ales - 20161205202856.mp4>)

In [14]:
process_video_source(user, video_source)

43
ales - 20161205202856.mp4 9 True 6.0 3
ales - 20161205202856.mp4 12 True 9.0 3
ales - 20161205202856.mp4 15 True 12.0 4
ales - 20161205202856.mp4 19 True 16.0 5
ales - 20161205202856.mp4 29 True 26.0 3
ales - 20161205202856.mp4 32 True 29.0 4
ales - 20161205202856.mp4 36 True 33.0 5
ales - 20161205202856.mp4 43 False 40.0 5
49 3 46 46
ales - 20161205202856.mp4 68 True 52.0 5
ales - 20161205202856.mp4 73 True 57.0 3
ales - 20161205202856.mp4 76 False 60.0 5
82 62 20 66
ales - 20161205202856.mp4 90 True 72.0 4
ales - 20161205202856.mp4 94 True 76.0 3
ales - 20161205202856.mp4 97 True 79.0 3
ales - 20161205202856.mp4 100 False 82.0 5
106 84 22 88
ales - 20161205202856.mp4 114 True 94.0 5
ales - 20161205202856.mp4 123 True 103.0 3
ales - 20161205202856.mp4 126 True 106.0 5
ales - 20161205202856.mp4 131 True 111.0 5
ales - 20161205202856.mp4 139 False 119.0 5
145 108 37 125
ales - 20161205202856.mp4 173 True 131.0 3
ales - 20161205202856.mp4 176 True 134.0 3
ales - 20161205202856.mp4 179





  0%|          | 0/5491 [00:00<?, ?it/s]



  1%|          | 66/5491 [00:00<00:08, 657.33it/s]



  3%|▎         | 156/5491 [00:00<00:07, 714.24it/s]



  5%|▍         | 250/5491 [00:00<00:06, 769.53it/s]



  6%|▌         | 334/5491 [00:00<00:06, 768.05it/s]



  8%|▊         | 419/5491 [00:00<00:06, 790.47it/s]



  9%|▉         | 506/5491 [00:00<00:06, 810.69it/s]



 11%|█         | 599/5491 [00:00<00:05, 840.63it/s]



 12%|█▏        | 684/5491 [00:00<00:05, 837.27it/s]



 14%|█▍        | 767/5491 [00:00<00:05, 834.41it/s]



 16%|█▌        | 855/5491 [00:01<00:05, 842.18it/s]



 17%|█▋        | 938/5491 [00:01<00:05, 820.81it/s]



 19%|█▊        | 1020/5491 [00:01<00:06, 648.42it/s]



 20%|██        | 1110/5491 [00:01<00:06, 706.36it/s]



 22%|██▏       | 1200/5491 [00:01<00:05, 753.20it/s]



 23%|██▎       | 1289/5491 [00:01<00:05, 788.99it/s]



 25%|██▌       | 1384/5491 [00:01<00:04, 829.93it/s]



 27%|██▋       | 1471/5491 [00:01<00:06, 661.57it/s]



 28%|██▊    

[MoviePy] Done.
[MoviePy] Writing video /home/tennistat/webapps/tennistat2/backend/media/user_12/video_sources/processed/20161205202856.mp4






  0%|          | 0/7493 [00:00<?, ?it/s]



  0%|          | 3/7493 [00:00<06:21, 19.62it/s]



  0%|          | 6/7493 [00:00<06:06, 20.43it/s]



  0%|          | 9/7493 [00:00<05:45, 21.69it/s]



  0%|          | 12/7493 [00:00<05:46, 21.62it/s]



  0%|          | 15/7493 [00:00<05:38, 22.06it/s]



  0%|          | 18/7493 [00:00<05:30, 22.60it/s]



  0%|          | 21/7493 [00:00<05:18, 23.47it/s]



  0%|          | 24/7493 [00:01<05:19, 23.35it/s]



  0%|          | 27/7493 [00:01<05:12, 23.88it/s]



  0%|          | 30/7493 [00:01<05:06, 24.37it/s]



  0%|          | 36/7493 [00:01<04:16, 29.07it/s]



  1%|          | 41/7493 [00:01<03:47, 32.77it/s]



  1%|          | 46/7493 [00:01<03:34, 34.74it/s]



  1%|          | 50/7493 [00:01<03:32, 34.98it/s]



  1%|          | 55/7493 [00:01<03:21, 36.86it/s]



  1%|          | 59/7493 [00:01<03:31, 35.13it/s]



  1%|          | 63/7493 [00:02<03:29, 35.48it/s]



  1%|          | 68/7493 [00:02<03:23, 36.53it/s]



 

[MoviePy] Done.
[MoviePy] >>>> Video ready: /home/tennistat/webapps/tennistat2/backend/media/user_12/video_sources/processed/20161205202856.mp4 



In [13]:
video_source.duration.total_seconds()

318.0